Going to train some NNs on the different delta data I guess.... not sure how I'm gonna split it up yet but lets get started

In [1]:
#imports
import uproot as ur
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import vector as v

data_path = '/fast_scratch/atlas_images/v01-45/'

import atlas_mpl_style as ampl #makes matplotlib follow atlas formal style
ampl.use_atlas_style()

v.register_awkward()

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "2" #specify GPU
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import tensorflow as tf

# energyflow imports
import energyflow as ef
from energyflow.archs import PFN
from energyflow.utils import data_split

from sklearn.metrics import roc_auc_score, roc_curve

import graph_util as gu
import plot_util as pu
import matplotlib.pyplot as plt

2021-07-15 13:54:31.851806: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
branches = ['cluster_hitsTruthIndex', 'cluster_hitsTruthE', 'truthPartPdgId', 'truthPartPt','truthPartEta', 
            'truthPartPhi', 'truthPartPt', 'truthPartMass']

events = ur.open(data_path+'delta_medium.root')

tree = events['EventTree']

In [3]:
the_branches = tree.arrays(expressions=branches)

To do this, I need to first set up the different selections, then do the data processing, (I guess the other way around works too? Okay I'm gonna change the convert_sets_delta file to also return truthID, then can do the selections on the pre-made data, then can run the models

In [4]:
tree.show()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
runNumber            | int32_t                  | AsDtype('>i4')
eventNumber          | int64_t                  | AsDtype('>i8')
lumiBlock            | int32_t                  | AsDtype('>i4')
coreFlags            | uint32_t                 | AsDtype('>u4')
mcEventNumber        | int32_t                  | AsDtype('>i4')
mcChannelNumber      | int32_t                  | AsDtype('>i4')
mcEventWeight        | float                    | AsDtype('>f4')
nTruthPart           | int32_t                  | AsDtype('>i4')
G4PreCalo_n_EM       | int32_t                  | AsDtype('>i4')
G4PreCalo_E_EM       | float                    | AsDtype('>f4')
G4PreCalo_n_Had      | int32_t                  | AsDtype('>i4')
G4PreCalo_E_Had      | float                    | AsDtype('>f4')
truthVertexX         | float                    | AsDtype(

In [6]:
import convert_sets_delta #okay so this doesn't line up, why?

Working on /fast_scratch/atlas_images/v01-45/delta_medium.root
Loading data


KeyboardInterrupt: 

In [5]:
clus_E = tree.arrays('cluster_E')
truth_ID = tree.arrays('truthPartPdgId')

In [6]:
len(clus_E)

200000

In [7]:
len(truth_ID)

200000

In [8]:
clus_e = gu.loadVectorBranchFlat('cluster_E', tree)

In [9]:
len(clus_e)

740871

In [10]:
truid = gu.loadVectorBranchFlat('truthPartPdgId', tree)

In [11]:
len(truid)

751874

In [12]:
#okay so its because they're being flattened and are different lengths... 
print(truid)

[2114 2112  111 ... 2224 2212  211]


In [13]:
print(clus_e)

[2.0940566  0.79894364 0.5638675  ... 0.63663745 0.3209459  1.641062  ]


In [14]:
truthe = tree.arrays('truthPartE')

In [15]:
len(truthe)

200000

In [16]:
trueflat = gu.loadVectorBranchFlat('truthPartE', tree)

In [17]:
len(trueflat)

751874

In [18]:
clusID = gu.loadVectorBranchFlat('cluster_cell_ID', tree)

ValueError: in ListOffsetArray64, cannot convert to RegularArray because subarray lengths are not regular

(https://github.com/scikit-hep/awkward-1.0/blob/1.4.0/src/cpu-kernels/awkward_ListOffsetArray_toRegularArray.cpp#L22)

In [ ]:
clusID

#### okay I think I see how to do it: make separate datasets for each???? so load it all up here with all the different branches I'll need, do the selection from the last notebook to separate all the types of deltas and apply it to each different branch that I need, then save those in separate files to call the convert sets on. Could even save them in the same file and have like a flag/indicator as to which delta each event is, so that I could run convert_sets on multiple different types of delta at once? okay yay

In [20]:
geotree = events['CellGeo']

In [21]:
geotree.show()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
cell_geo_ID          | std::vector<uint64_t>    | AsJagged(AsDtype('>u8'), he...
cell_geo_sampling    | std::vector<uint16_t>    | AsJagged(AsDtype('>u2'), he...
cell_geo_eta         | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
cell_geo_phi         | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
cell_geo_rPerp       | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
cell_geo_deta        | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
cell_geo_dphi        | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
cell_geo_volume      | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
cell_geo_sigma       | std::vector<float>       | AsJagged(AsDtype('>f4'), he...


In [22]:
cellgeoeta = geotree.arrays('cell_geo_eta')

In [23]:
len(cellgeoeta)

10

Okay so that confirms I don't have to mess with this stuff; recall this is all just its own thing, and it applies to any event, and is not event-specific, it just remains true for all events, it just describes the detector. okay.

In [24]:
rb = ['cluster_cell_ID', 'cluster_cell_E', 'cluster_Phi', 'cluster_Eta', 'cluster_Pt', 'truthPartPdgId',
               'cluster_E', 'cluster_ENG_CALIB_TOT'] #okay these are actually the only ones I need

In [25]:
brnsh = tree.arrays(expressions=rb)

In [26]:
#load up all the data we need
clus_cell_id = tree.arrays('cluster_cell_ID')
clus_cell_E = tree.arrays(rb[1])
clus_phi = tree.arrays(rb[2])
clus_eta = tree.arrays(rb[3])
clus_pt = tree.arrays(rb[4])
tru_id = tree.arrays(rb[5])
clus_E = tree.arrays(rb[6])
#I dont think I need calib?

In [27]:
tru_id = tree.arrays(expressions=rb[5])

In [28]:
#load up all data we need
tru_id = brnsh.truthPartPdgId
clus_cell_id = brnsh.cluster_cell_ID
clus_cell_E = brnsh.cluster_cell_E
clus_phi = brnsh.cluster_Phi
clus_eta = brnsh.cluster_Eta
clus_pt = brnsh.cluster_Pt
clus_E = brnsh.cluster_E

In [29]:
truth_ID, tru_id

(<Array [{truthPartPdgId: [2114, ... 211]}] type='200000 * {"truthPartPdgId": var...'>,
 <Array [[2114, 2112, 111, ... 2224, 2212, 211]] type='200000 * var * int32'>)

In [30]:
len(tru_id)

200000

In [31]:
np.array(tru_id[0])

array([2114, 2112,  111,   22,   22], dtype=int32)

In [32]:
#sort into particle types
ID1114 = []
cci1114 = []
ccE1114 = []
cph1114 = []
cet1114 = []
cpt1114 = []
cE1114 = []
ind1114 = []

IDn2214 = []
ccin2214 = []
ccEn2214 = []
cphn2214 = []
cetn2214 = []
cptn2214 = []
cEn2214 = []
indn2214 = []

IDn2114 = []
ccin2114 = []
ccEn2114 = []
cphn2114 = []
cetn2114 = []
cptn2114 = []
cEn2114 = []
indn2114 = []

IDn2224 = []
ccin2224 = []
ccEn2224 = []
cphn2224 = []
cetn2224 = []
cptn2224 = []
cEn2224 = []
indn2224 = []

ID2214 = []
cci2214 = []
ccE2214 = []
cph2214 = []
cet2214 = []
cpt2214 = []
cE2214 = []
ind2214 = []

ID2224 = []
cci2224 = []
ccE2224 = []
cph2224 = []
cet2224 = []
cpt2224 = []
cE2224 = []
ind2224 = []

IDn1114 = []
ccin1114 = []
ccEn1114 = []
cphn1114 = []
cetn1114 = []
cptn1114 = []
cEn1114 = []
indn1114 = []

ID2114 = []
cci2114 = []
ccE2114 = []
cph2114 = []
cet2114 = []
cpt2114 = []
cE2114 = []
ind2114 = []

for i in range(len(tru_id)):
    
    if (-2114 in tru_id[i]): 
        IDn2114.append(tru_id[i])
        ccin2114.append(clus_cell_id[i])
        ccEn2114.append(clus_cell_E[i])
        cphn2114.append(clus_phi[i])
        cetn2114.append(clus_eta[i])
        cptn2114.append(clus_pt[i])
        cEn2114.append(clus_E[i])
        indn2114.append(i)

    elif (-2224 in tru_id[i]):
        IDn2224.append(tru_id[i])
        ccin2224.append(clus_cell_id[i])
        ccEn2224.append(clus_cell_E[i])
        cphn2224.append(clus_phi[i])
        cetn2224.append(clus_eta[i])
        cptn2224.append(clus_pt[i])
        cEn2224.append(clus_E[i])
        indn2224.append(i)
        
    elif (2214 in tru_id[i]): 
        ID2214.append(tru_id[i])
        cci2214.append(clus_cell_id[i])
        ccE2214.append(clus_cell_E[i])
        cph2214.append(clus_phi[i])
        cet2214.append(clus_eta[i])
        cpt2214.append(clus_pt[i])
        cE2214.append(clus_E[i])
        ind2214.append(i)
        
    elif (2224 in tru_id[i]):
        ID2224.append(tru_id[i])
        cci2224.append(clus_cell_id[i])
        ccE2224.append(clus_cell_E[i])
        cph2224.append(clus_phi[i])
        cet2224.append(clus_eta[i])
        cpt2224.append(clus_pt[i])
        cE2224.append(clus_E[i])
        ind2224.append(i)
        
    elif (-1114 in tru_id[i]): 
        IDn1114.append(tru_id[i])
        ccin1114.append(clus_cell_id[i])
        ccEn1114.append(clus_cell_E[i])
        cphn1114.append(clus_phi[i])
        cetn1114.append(clus_eta[i])
        cptn1114.append(clus_pt[i])
        cEn1114.append(clus_E[i])
        indn1114.append(i)
        
    elif (2114 in tru_id[i]):
        ID2114.append(tru_id[i])
        cci2114.append(clus_cell_id[i])
        ccE2114.append(clus_cell_E[i])
        cph2114.append(clus_phi[i])
        cet2114.append(clus_eta[i])
        cpt2114.append(clus_pt[i])
        cE2114.append(clus_E[i])
        ind2114.append(i)
        
    elif (1114 in tru_id[i]): 
        ID1114.append(tru_id[i])
        cci1114.append(clus_cell_id[i])
        ccE1114.append(clus_cell_E[i])
        cph1114.append(clus_phi[i])
        cet1114.append(clus_eta[i])
        cpt1114.append(clus_pt[i])
        cE1114.append(clus_E[i])
        ind1114.append(i)
        
    elif (-2214 in tru_id[i]):
        IDn2214.append(tru_id[i])
        ccin2214.append(clus_cell_id[i])
        ccEn2214.append(clus_cell_E[i])
        cphn2214.append(clus_phi[i])
        cetn2214.append(clus_eta[i])
        cptn2214.append(clus_pt[i])
        cEn2214.append(clus_E[i])
        indn2214.append(i)

In [33]:
# I don't think I can just make a root file..... so this goes into the delta conv_sets I guess

In [34]:
len(np.array(ak.flatten(tru_id))), len(np.array(ak.flatten(clus_eta)))

(751874, 740871)

In [35]:
len(tru_id), len(clus_eta) #so the selection needs to happen here

(200000, 200000)

In [36]:
#do I make a flag that says what kind it is?????

flag = np.zeros(len(tru_id))
flag[indn2214] = -2214
flag[ind2214] = 2214
flag[ind1114] = 1114
flag[indn1114] = -1114
flag[ind2114] = 2114
flag[indn2114] = -2114
flag[ind2224] = 2224
flag[indn2224] = -2224

In [37]:
flag

array([ 2114., -2224.,  2224., ..., -2114.,  2114.,  2224.])

In [38]:
x = flag == 0

In [39]:
flag[x] #okay so its all covered now

array([], dtype=float64)

In [40]:
len(flag)

200000

In [41]:
len(clus_eta)

200000

In [42]:
clus_eta[3]

<Array [-1.92, -1.87] type='2 * float32'>

In [43]:
np.repeat(flag[0], 3)

array([2114., 2114., 2114.])

In [44]:
#this is really janky but here goes nothing:
longflag = []
for i in range(len(clus_eta)):
    longflag.append(np.repeat(flag[i], len(clus_eta[i]))) #hopefuly that works lol

In [45]:
len(np.array(ak.flatten(clus_eta))), len(longflag)

(740871, 200000)

In [46]:
longflag[i]

array([2224.])

In [47]:
np.ndarray.flatten(np.array(longflag))

/tmp/ipykernel_51992/2564729344.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.ndarray.flatten(np.array(longflag))


array([array([2114., 2114., 2114., 2114.]), array([-2224.]),
       array([2224., 2224., 2224., 2224., 2224., 2224., 2224., 2224., 2224.,
              2224., 2224., 2224., 2224., 2224., 2224., 2224., 2224., 2224.,
              2224., 2224., 2224., 2224.])                                  ,
       ..., array([-2114., -2114., -2114., -2114.]),
       array([2114., 2114., 2114.]), array([2224.])], dtype=object)

In [48]:
#ACTUALLY who cares about longflag, but flag is good because its uselful to make a true/false with which to make a
#mask of the 200000 length data at the beginning of the data processing. pass the 200000 mask to the loading 
#functions in gu!!! so close!!! so easy!!! ugh!!!

In [49]:
# so to do it, pass it the mask; just run convsets.doit(mask) after loading up/selecting the mask using this:

idbranch = tree.arrays('truthPartPdgId')
truth_ID = idbranch.truthPartPdgId

#just need to get the indices of each of them - I should probably just do this with like np.where or smtn but for now this works heheheh :(
ID1114 = []
IDn2214 = []    
IDn2114 = []    
IDn2224 = []   
ID2214 = []
ID2224 = []
IDn1114 = []
ID2114 = []
for i in range(len(truth_ID)):
    if (-2114 in tru_id[i]): 
        IDn2114.append(i)
    elif (-2224 in tru_id[i]):
        IDn2224.append(i)
    elif (2214 in tru_id[i]): 
        ID2214.append(i)
    elif (2224 in tru_id[i]):
        ID2224.append(i)
    elif (-1114 in tru_id[i]): 
        IDn1114.append(i)
    elif (2114 in tru_id[i]):
        ID2114.append(i)
    elif (1114 in tru_id[i]): 
        ID1114.append(i)
    elif (-2214 in tru_id[i]):
        IDn2214.append(i)

#this is literally still so janky but whatever
flag = np.zeros(len(tru_id))
flag[IDn2214] = -2214
flag[ID2214] = 2214
flag[ID1114] = 1114
flag[IDn1114] = -1114
flag[ID2114] = 2114
flag[IDn2114] = -2114
flag[ID2224] = 2224
flag[IDn2224] = -2224

In [50]:
#then choose the mask from there: eg
mask = (flag == 2214) | (flag == -2214) | (flag == 2114) | (flag == -2114)

In [51]:
flag == 2214 | -2214

array([False, False, False, ..., False, False, False])

In [52]:
flag

array([ 2114., -2224.,  2224., ..., -2114.,  2114.,  2224.])

In [1]:
longflag

NameError: name 'longflag' is not defined

In [53]:
mask #okay cool I think this works then...!

array([ True, False, False, ...,  True,  True, False])

In [54]:
import graph_util_splitdelta as gusd

In [55]:
cell_e  = gusd.loadArrayBranchFlat('cluster_cell_E', tree, 2000, mask)

In [56]:
len(clus_E), len(clus_cell_E[mask]), len(truth_ID[mask])

(200000, 100000, 100000)

In [57]:
len(cell_e[0])

2000

In [60]:
clu_cell_e = clus_cell_E[mask]
clus_E_msk = clus_E[mask]
masktruID = truth_ID[mask]

In [61]:
len(clu_cell_e), len(masktruID), len(clus_E_msk)

(100000, 100000, 100000)

In [62]:
len(clus_E_msk[0]), len(masktruID[0]) #like how bruh omfg

(4, 5)

In [63]:
print(clus_E_msk[0] > .5)

[True, True, True, False]


In [64]:
clus_E_msk.shape

AttributeError: no field named 'shape'

(https://github.com/scikit-hep/awkward-1.0/blob/1.4.0/src/awkward/highlevel.py#L1127)

In [86]:
len(np.ravel(cell_e))

712852000

In [65]:
r = clus_E_msk > .5

In [70]:
len(clus_E_msk[r])

100000

In [71]:
masktruID[r] #yeah so this won't work, so just do it the new way of just selecting all 8 one by one.....

ValueError: in ListArray64 attempting to get 3, index out of range

(https://github.com/scikit-hep/awkward-1.0/blob/1.4.0/src/cpu-kernels/awkward_ListArray_getitem_jagged_apply.cpp#L46)

In [92]:
import convert_sets_splitdelta as cs

In [93]:
cs.doit(mask) #okay so need to figure out how to get the truth_id to the data_split function

Working on /fast_scratch/atlas_images/v01-45/delta_medium.root
Loading data
Normalizing


/home/kaelaekd/start_tf/LCStudies/classifier/convert_sets_splitdelta.py:61: RuntimeWarning: divide by zero encountered in log
  cell_e = np.nan_to_num(np.log(cell_e), posinf = 0, neginf=0)


Writing out


TypeError: no implementation found for 'numpy.savez' on types that implement __array_function__: [<class 'numpy.ndarray'>, <class 'awkward.highlevel.Array'>]

here's the thing tho: no idea how to give it the e_mask. no idea. it is needed right? it must be...... how will it go through split_data? they're not the same lengths anymore again.... I think I did need that flag or whatever afterall tbh? maybe as well? 